# Evolution of Einstein's equations

# Initial data , setting length of the domain and discretization


In [9]:
# Parameters
m = 1
A = 0.1124921875#0.125
run = 1

1

In [10]:
res=m;
N=2.0^m*50.0*7.5#2.0^m*1000.0;#2.0^m*500.0;#N=2.0^m*500.0#2.0^m*100.0;
Xf=1.0;

dx=Xf/N;
dt=0.8*round(dx,digits=10);
Nt=2.0^m*50.0*7.5#100.0*2^m*10
Tf=Nt*dt; #final time
print(Tf)

0.79999998

In [11]:
print(N)

750.0

In [12]:
println("dx =", dx)
println("dt =", dt)
println("dx/dt =", dx/dt)

dx =0.0013333333333333333
dt =0.00106666664
dx/dt =1.2500000312500008


In [5]:
# Setting RESOLUTION

global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res$res"
#global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res"

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res1"

# Grid

In [6]:
ori=0.0;
initX1 = nothing
initX1=range(ori, stop=Xf, step=dx);

In [7]:
initX = range(round(ori-3.0*dx,digits=10), stop=Xf+3.0*dx, step=dx)

L=length(initX);
println("step size is  ", dx)

#T=range(dt,stop=Tf,step=dt)
#T=range(0,stop=Tf,step=dt)
println("the time step is ", dt)

step size is  0.0013333333333333333
the time step is 0.00106666664


In [8]:
include("./ScalarField.jl");

# Initial data of m β Ψ and Ψ,x

In [149]:
using PyCall   
using Interpolations
using TickTock
using Dierckx

tick()
scipy = pyimport("scipy")
scipyinterpolate = pyimport("scipy.interpolate")


####

initm=zeros(L);
initbeta=zeros(L);
initpsi=zeros(L);
initderpsi=zeros(L);

state_array=[initm initbeta initpsi initderpsi initX];

#PSI
r0=0.7#0.3
sigma=0.3
initpsi[4:L-3] = init_gaussian(initX1,r0,sigma,A)

state_array[:,3] = initpsi
state_array = ghost(state_array)

#PSI,X FROM PSI
"""for i in 4:L-3
    initderpsi[i]=Der(state_array[:,:],i,3,initX)
end

state_array[:,4] = initderpsi
state_array = ghost(state_array)"""

#new
initderpsi[4:L-3] = init_gaussian_der(initX1,r0,sigma,A)
state_array[:,4] = initderpsi
state_array=ghost(state_array)

####


psi_func = Spline1D(initX[4:L-3], state_array[4:L-3,3],  k=4)
derpsi_func = Spline1D(initX[4:L-3], initderpsi[4:L-3],  k=4)#new

funcs = [psi_func, derpsi_func];

tock()

[ Info:  started timer at: 2023-04-05T00:20:39.609
[ Info:          0.218659256s: 218 milliseconds


In [150]:
#BETA
beta0=0
initbeta[4:L-3]=rk4wrapper(SFconstraint_beta,beta0,initX1,0,funcs)
state_array[:,2]=initbeta;
state_array=ghost(state_array);


#M
m0=0
initm[4:L-3]=rk4wrapper(SFconstraint_m,m0,initX1,0,funcs)
state_array[:,1]=initm;
state_array = ghost(state_array);

In [151]:
"""using Plots #showing initial data and the initial status of the horizon finder
plot(initX, state_array[:,1:4], xaxis="x",title="Initial Data", labels=["m" "beta" "psi" "psi,x"])"""

"using Plots #showing initial data and the initial status of the horizon finder\nplot(initX, state_array[:,1:4], xaxis=\"x\",title=\"Initial Data\", labels=[\"m\" \"beta\" \"psi\" \"psi,x\"])"

# Defining the initial state array for the evolution m β Ψ and Ψ,x

In [152]:
using CSV, Tables
#CSV.write(dir*"/time_step0.csv", Tables.table(transpose(Matrix(state_array))), writeheader=false)
CSV.write(dir*"/time_step0.csv", Tables.table(state_array), writeheader=false)

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res1/time_step0.csv"

In [153]:
timestep=0
criticality=0.0
explode=0.0
evol_stats = [run criticality A sigma r0 timestep explode]
monitor_ratio = zeros(L)
CSV.write("/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv", Tables.table(evol_stats), writeheader=true, header=["run", "criticality", "A", "sigma", "r0", "timestep", "explode"])

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv"

In [154]:
ginit=dt_scale(initX,state_array[:,1],state_array[:,2],dx)
println(update_dt(initX,state_array[:,1],state_array[:,2],dx,ginit)/dt)


1.0000000250000007


 ## Time evolution

fixed grid

In [155]:
finaltime=1#length(T)-1
T = timeevolution(state_array,finaltime,dir,dt,run)

iteration 1 dt is 0.0010666666666666667, time of iteration is 0.0010666666666666667
iteration 2 dt is 0.0010641736292105151, time of iteration is 0.002130840295877182
iteration 3 dt is 0.0010616929152979517, time of iteration is 0.003192533211175134
iteration 4 dt is 0.001059224422927336, time of iteration is 0.00425175763410247
iteration 5 dt is 0.0010567680513026052, time of iteration is 0.005308525685405075
iteration 6 dt is 0.0010543237007643333, time of iteration is 0.006362849386169408
iteration 7 dt is 0.0010518912727734684, time of iteration is 0.007414740658942877
iteration 8 dt is 0.0010494706698953792, time of iteration is 0.008464211328838256
iteration 9 dt is 0.001047061795784163, time of iteration is 0.009511273124622419
iteration 10 dt is 0.0010446645551672033, time of iteration is 0.010555937679789622
iteration 11 dt is 0.0010422788538300355, time of iteration is 0.011598216533619657
iteration 12 dt is 0.0010399045986014298, time of iteration is 0.012638121132221087
ite

iteration 100 dt is 0.0008670579229453, time of iteration is 0.09598989832526689
iteration 101 dt is 0.0008654245208011439, time of iteration is 0.09685532284606803
iteration 102 dt is 0.0008637971251157288, time of iteration is 0.09771911997118375
iteration 103 dt is 0.0008621756977860385, time of iteration is 0.0985812956689698
iteration 104 dt is 0.0008605602010398575, time of iteration is 0.09944185587000966
iteration 105 dt is 0.0008589505974321431, time of iteration is 0.1003008064674418
iteration 106 dt is 0.0008573468498414323, time of iteration is 0.10115815331728323
iteration 107 dt is 0.0008557489214663124, time of iteration is 0.10201390223874954
iteration 108 dt is 0.0008541567758219236, time of iteration is 0.10286805901457147
iteration 109 dt is 0.0008525703767365077, time of iteration is 0.10372062939130798
iteration 110 dt is 0.0008509896883480244, time of iteration is 0.10457161907965601
iteration 111 dt is 0.0008494146751007782, time of iteration is 0.105421033754756

iteration 200 dt is 0.0007283596427241492, time of iteration is 0.1753147145549581
iteration 201 dt is 0.0007271804348651074, time of iteration is 0.1760418949898232
iteration 202 dt is 0.0007260046185323782, time of iteration is 0.17676789960835557
iteration 203 dt is 0.0007248321763455992, time of iteration is 0.17749273178470115
iteration 204 dt is 0.000723663091047111, time of iteration is 0.17821639487574825
iteration 205 dt is 0.0007224973455008728, time of iteration is 0.17893889222124912
iteration 206 dt is 0.0007213349226913885, time of iteration is 0.1796602271439405
iteration 207 dt is 0.000720175805722634, time of iteration is 0.18038040294966315
iteration 208 dt is 0.0007190199778170083, time of iteration is 0.18109942292748016
iteration 209 dt is 0.0007178674223142928, time of iteration is 0.18181729034979446
iteration 210 dt is 0.0007167181226706077, time of iteration is 0.18253400847246506
iteration 211 dt is 0.0007155720624574059, time of iteration is 0.183249580534922

iteration 398 dt is 0.0005445485176935708, time of iteration is 0.29982037701873304
iteration 399 dt is 0.0005438106363029606, time of iteration is 0.30036418765503603
iteration 400 dt is 0.0005430741967176546, time of iteration is 0.3009072618517537
iteration 401 dt is 0.0005423391937220239, time of iteration is 0.30144960104547575
iteration 402 dt is 0.0005416056221278909, time of iteration is 0.30199120666760365
iteration 403 dt is 0.000540873476774358, time of iteration is 0.302532080144378
iteration 404 dt is 0.0005401427525276376, time of iteration is 0.3030722228969056
iteration 405 dt is 0.0005394134442808689, time of iteration is 0.3036116363411865
iteration 406 dt is 0.000538685546953944, time of iteration is 0.30415032188814045
iteration 407 dt is 0.0005379590554933413, time of iteration is 0.3046882809436338
iteration 408 dt is 0.0005372339648719516, time of iteration is 0.30522551490850575
iteration 409 dt is 0.000536510270088911, time of iteration is 0.30576202517859463
i

iteration 497 dt is 0.0004777664080669429, time of iteration is 0.3502927308254121
iteration 498 dt is 0.00047714968759024544, time of iteration is 0.3507698805130024
iteration 499 dt is 0.00047653400227068486, time of iteration is 0.3512464145152731
iteration 500 dt is 0.00047591934892390804, time of iteration is 0.351722333864197
iteration 501 dt is 0.0004753057243804933, time of iteration is 0.35219763958857747
iteration 502 dt is 0.0004746931254858601, time of iteration is 0.35267233271406334
iteration 503 dt is 0.0004740815491001797, time of iteration is 0.3531464142631635
iteration 504 dt is 0.00047347099209828875, time of iteration is 0.3536198852552618
iteration 505 dt is 0.00047286145136961116, time of iteration is 0.3540927467066314
iteration 506 dt is 0.000472252923818061, time of iteration is 0.35456499963044946
iteration 507 dt is 0.0004716454063619721, time of iteration is 0.3550366450368114
iteration 508 dt is 0.00047103889593400045, time of iteration is 0.35550768393274

...

iteration 4075 dt is 7.850372021402983e-6, time of iteration is 0.7155096536889761
iteration 4076 dt is 7.841634968208438e-6, time of iteration is 0.7155174953239444
iteration 4077 dt is 7.832915360272203e-6, time of iteration is 0.7155253282393046
iteration 4078 dt is 7.824213276655988e-6, time of iteration is 0.7155331524525812
iteration 4079 dt is 7.815528775676107e-6, time of iteration is 0.7155409679813569
iteration 4080 dt is 7.806861893886122e-6, time of iteration is 0.7155487748432507
iteration 4081 dt is 7.798212645119102e-6, time of iteration is 0.7155565730558958
iteration 4082 dt is 7.789581019589035e-6, time of iteration is 0.7155643626369154
iteration 4083 dt is 7.7809669830532e-6, time of iteration is 0.7155721436038984
iteration 4084 dt is 7.772370476037856e-6, time of iteration is 0.7155799159743744
iteration 4085 dt is 7.763791413125438e-6, time of iteration is 0.7155876797657875
iteration 4086 dt is 7.755229682305226e-6, time of iteration is 0.7155954349954698
iterat

iteration 4175 dt is 6.977717745311543e-6, time of iteration is 0.7162517743895541
iteration 4176 dt is 6.967257954380356e-6, time of iteration is 0.7162587416475085
iteration 4177 dt is 6.956715662991049e-6, time of iteration is 0.7162656983631714
iteration 4178 dt is 6.946089060522684e-6, time of iteration is 0.7162726444522319
iteration 4179 dt is 6.935376312427422e-6, time of iteration is 0.7162795798285444
iteration 4180 dt is 6.924575559612971e-6, time of iteration is 0.716286504404104
iteration 4181 dt is 6.913684917849177e-6, time of iteration is 0.7162934180890219
iteration 4182 dt is 6.9027024772016264e-6, time of iteration is 0.7163003207914991
iteration 4183 dt is 6.891626301499621e-6, time of iteration is 0.7163072124178006
iteration 4184 dt is 6.880454427836972e-6, time of iteration is 0.7163140928722285
iteration 4185 dt is 6.869184866116165e-6, time of iteration is 0.7163209620570946
iteration 4186 dt is 6.857815598633879e-6, time of iteration is 0.7163278198726932
iter

iteration 4275 dt is 5.159305973229928e-6, time of iteration is 0.7168738911463228
iteration 4276 dt is 5.129618818833218e-6, time of iteration is 0.7168790207651417
iteration 4277 dt is 5.099654706056573e-6, time of iteration is 0.7168841204198477
iteration 4278 dt is 5.069413147223209e-6, time of iteration is 0.716889189832995
iteration 4279 dt is 5.038893700179345e-6, time of iteration is 0.7168942287266952
iteration 4280 dt is 5.008095972811408e-6, time of iteration is 0.716899236822668
iteration 4281 dt is 4.977019628270771e-6, time of iteration is 0.7169042138422963
iteration 4282 dt is 4.945664390939662e-6, time of iteration is 0.7169091595066872
iteration 4283 dt is 4.914030053166301e-6, time of iteration is 0.7169140735367404
iteration 4284 dt is 4.882116482787852e-6, time of iteration is 0.7169189556532232
iteration 4285 dt is 4.849923631470295e-6, time of iteration is 0.7169238055768546
iteration 4286 dt is 4.817451543864898e-6, time of iteration is 0.7169286230283985
iterat

iteration 4375 dt is 1.5090247027940927e-6, time of iteration is 0.7172066356818494
iteration 4376 dt is 1.4793909908395976e-6, time of iteration is 0.7172081150728402
iteration 4377 dt is 1.450083204449184e-6, time of iteration is 0.7172095651560446
iteration 4378 dt is 1.421099088254944e-6, time of iteration is 0.7172109862551329
iteration 4379 dt is 1.3924362614143354e-6, time of iteration is 0.7172123786913943
iteration 4380 dt is 1.364092239983786e-6, time of iteration is 0.7172137427836343
iteration 4381 dt is 1.336064458706322e-6, time of iteration is 0.717215078848093
iteration 4382 dt is 1.3083502921621616e-6, time of iteration is 0.7172163871983852
iteration 4383 dt is 1.2809470752016765e-6, time of iteration is 0.7172176681454604
iteration 4384 dt is 1.2538521225814095e-6, time of iteration is 0.717218921997583
iteration 4385 dt is 1.2270627477394559e-6, time of iteration is 0.7172201490603307
iteration 4386 dt is 1.20057628066062e-6, time of iteration is 0.7172213496366113


1×7 Matrix{Float64}:
 1.0  0.112492  0.3  0.7  4462.0  0.0  1.0

## Movie of the evolution

In [ ]:
using Plots, CSV, DataFrames
import GR
anim = @animate for n in 1:10:740#length(T)-1
    
    time_step=T[n+1]-T[n];
    IJulia.clear_output(true)
    
    functions3=DataFrame(CSV.File(dir*"/time_step$n.csv",header=false, delim=","))
    plot()
        plot1=functions3.Column1[4:L-3];
        plot2=functions3.Column2[4:L-3];
        plot3=functions3.Column3[4:L-3];
        plot4=functions3.Column4[4:L-3];
        plot5=functions3.Column5[4:L-3];
    d=round(T[n],digits=1)
    #plot!(X1, [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],ylim=[-0.1,0.1], xlim=[0, 0.1]) |> display 
    plot!(initX[4:L-3], [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],ylim=[-1,1])#, xlim=[0, 0.1]) |> display 
    
    end

gif(anim, "anim_res$res.gif", fps = 1)

In [132]:
dir

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res1"